In [1]:
# packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [8]:
def calc_jobzone_dist_emp(JEDI_output,this_state):
    onet_jobzones = pd.read_excel(r"../ONET/ONET_Job_Zones_All_States.xlsx")
    state_name = pd.read_excel(r"../Census/State_Abbr.xlsx")
    
    JEDI_output_state = JEDI_output.query("state==@this_state") # state abbreviations
    JEDI_output_state = JEDI_output_state.set_index('i')     
    JEDI_output_state = JEDI_output_state.drop(columns="state")
    JEDI_output_state = JEDI_output_state.transpose()
    all_years = JEDI_output_state.columns
    JEDI_output_state["Type"] = "Total"
    
    state_full = state_name.query("Abbr==@this_state").iloc[0,0]
    onet_jobzones_state = onet_jobzones.query("State == @state_full")
    onet_jobzones_state = onet_jobzones_state.set_index("JEDI_Industries")
    onet_jobzones_state = onet_jobzones_state.drop(columns="State")
    
    # Separate everything by each job zone
    for zone in [1,2,3,4,5]:
        this_zone = onet_jobzones_state[zone]
        this_zone_calc = pd.merge(JEDI_output_state,this_zone, left_index=True,right_index=True)
        for year in JEDI_output_state.columns[:-1]:
            this_zone_calc[year] = this_zone_calc[year] * this_zone_calc[zone]/100
        this_zone_calc["Type"] = zone
        this_zone_calc = this_zone_calc.drop(columns = zone)

        if zone == 1:
            all_zones_calc = this_zone_calc
        else:
            all_zones_calc = all_zones_calc.append(this_zone_calc)
    
    return all_zones_calc